# MongoDB As A Toolbox For Agentic Systems

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/tools/tools/function_calling_mongodb_as_a_toolbox.ipynb)

In [1]:
!pip install --quiet openai pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [2]:
import os
import json
import getpass

OPENAI_API_KEY = getpass.getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

MONGO_URI = getpass.getpass("Enter MongoDB URI: ")
os.environ["MONGO_URI"] = MONGO_URI

GPT_MODEL = "gpt-4o"

OpenAI API Key: ··········
Enter MongoDB URI: ··········


In [3]:
import openai

client = openai.OpenAI()

## Define MongoDB Tool Decorator

In [4]:
import pymongo

# Get MongoClient
mongo_client = pymongo.MongoClient(MONGO_URI)

# Get database
db = mongo_client['function_calling_db']

# Get collection
tools_collection = db['tools']

In [5]:
import inspect
from functools import wraps
from typing import get_type_hints


def get_embedding(text, model="text-embedding-3-small"):
  text = text.replace("\n", " ")
  return client.embeddings.create(input = [text], model=model).data[0].embedding

def mongodb_toolbox(collection=tools_collection):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)

        # Generate tool definition
        signature = inspect.signature(func)
        docstring = inspect.getdoc(func) or ""
        type_hints = get_type_hints(func)

        tool_def = {
            "name": func.__name__,
            "description": docstring.strip(),
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }

        for param_name, param in signature.parameters.items():
            if param.kind == inspect.Parameter.VAR_POSITIONAL or param.kind == inspect.Parameter.VAR_KEYWORD:
                continue

            param_type = type_hints.get(param_name, type(None))
            json_type = "string"  # Default to string
            if param_type in (int, float):
                json_type = "number"
            elif param_type == bool:
                json_type = "boolean"

            tool_def["parameters"]["properties"][param_name] = {
                "type": json_type,
                "description": f"Parameter {param_name}"
            }

            if param.default == inspect.Parameter.empty:
                tool_def["parameters"]["required"].append(param_name)

        tool_def["parameters"]["additionalProperties"] = False

        # Store in MongoDB
        vector = get_embedding(tool_def["description"])
        tool_doc = {
            **tool_def,
            "embedding": vector
        }
        collection.update_one({"name": func.__name__}, {"$set": tool_doc}, upsert=True)

        return wrapper
    return decorator

In [6]:
def vector_search(user_query, collection):
  """
  Perform a vector search in the MongoDB collection based on the user query.

  Args:
  user_query (str): The user's query string.
  collection (MongoCollection): The MongoDB collection to search.

  Returns:
  list: A list of matching documents.
  """

  # Generate embedding for the user query
  query_embedding = get_embedding(user_query)

  if query_embedding is None:
    return "Invalid query or embedding generation failed."

  # Define the vector search pipeline
  vector_search_stage = {
    "$vectorSearch": {
      "index": "vector_index",
      "queryVector": query_embedding,
      "path": "embedding",
      "numCandidates": 150,  # Number of candidate matches to consider
      "limit": 2  # Return top 5 matches
    }
  }

  unset_stage = {
    "$unset": "embedding"  # Exclude the 'embedding' field from the results
  }


  pipeline = [vector_search_stage, unset_stage]

  # Execute the search
  results = collection.aggregate(pipeline)
  return list(results)

In [7]:
import random
from datetime import datetime

@mongodb_toolbox()
def shout(statement: str) -> str:
  """
  Convert a statement to uppercase letters to emulate shouting. Use this when a user wants to emphasize something strongly or when they explicitly ask to 'shout' something..

  """
  return statement.upper()

@mongodb_toolbox()
def get_weather(location: str, unit: str = "celsius") -> str:
    """
    Get the current weather for a specified location.
    Use this when a user asks about the weather in a specific place.

    :param location: The name of the city or location to get weather for.
    :param unit: The temperature unit, either 'celsius' or 'fahrenheit'. Defaults to 'celsius'.
    :return: A string describing the current weather.
    """
    conditions = ["sunny", "cloudy", "rainy", "snowy"]
    temperature = random.randint(-10, 35)

    if unit.lower() == "fahrenheit":
        temperature = (temperature * 9/5) + 32

    condition = random.choice(conditions)
    return f"The weather in {location} is currently {condition} with a temperature of {temperature}°{'C' if unit.lower() == 'celsius' else 'F'}."

@mongodb_toolbox()
def get_stock_price(symbol: str) -> str:
    """
    Get the current stock price for a given stock symbol.
    Use this when a user asks about the current price of a specific stock.

    :param symbol: The stock symbol to look up (e.g., 'AAPL' for Apple Inc.).
    :return: A string with the current stock price.
    """
    price = round(random.uniform(10, 1000), 2)
    return f"The current stock price of {symbol} is ${price}."

@mongodb_toolbox()
def get_current_time(timezone: str = "UTC") -> str:
    """
    Get the current time for a specified timezone.
    Use this when a user asks about the current time in a specific timezone.

    :param timezone: The timezone to get the current time for. Defaults to 'UTC'.
    :return: A string with the current time in the specified timezone.
    """
    current_time = datetime.utcnow().strftime("%H:%M:%S")
    return f"The current time in {timezone} is {current_time}."


In [8]:
def populate_tools(search_results):
  """
  Populate the tools array based on the results from the vector search.

  Args:
  search_results (list): The list of documents returned from the vector search.

  Returns:
  list: A list of tool definitions in the format required by the OpenAI API.
  """
  tools = []
  for result in search_results:
    tool = {
      "type": "function",
      "function": {
        "name": result["name"],
        "description": result["description"],
        "parameters": result["parameters"]
      }
    }
    tools.append(tool)
  return tools

In [9]:
user_query = "Hi, can you shout the statement: We are there"

In [10]:
tools_related_to_user_query = vector_search(user_query, tools_collection)

In [11]:
tools = populate_tools(tools_related_to_user_query)

In [12]:
import pprint

pprint.pprint(tools)

[{'function': {'description': 'Convert a statement to uppercase letters to '
                              'emulate shouting. Use this when a user wants to '
                              'emphasize something strongly or when they '
                              "explicitly ask to 'shout' something..",
               'name': 'shout',
               'parameters': {'additionalProperties': False,
                              'properties': {'statement': {'description': 'Parameter '
                                                                          'statement',
                                                           'type': 'string'}},
                              'required': ['statement'],
                              'type': 'object'}},
  'type': 'function'},
 {'function': {'description': 'Get the current stock price for a given stock '
                              'symbol.\n'
                              'Use this when a user asks about the current '
                      

In [13]:
messages = [
  {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
  {"role": "user", "content": user_query}
]

In [14]:
response = openai.chat.completions.create(
  model=GPT_MODEL,
  messages=messages,
  tools=tools,
)

In [15]:
# Append the message to messages list
response_message = response.choices[0].message
messages.append(response_message)

print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_U7kkN9X2ohVasUD5ReOaRwcG', function=Function(arguments='{"statement":"We are there"}', name='shout'), type='function')])


In [16]:
import json

# Step 2: determine if the response from the model includes a tool call.
tool_calls = response_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)
    tool_call = tool_calls[0]
    tool_call_id = tool_call.id
    tool_function_name = tool_call.function.name

    print(f"Debug - Tool call received: {tool_function_name}")
    print(f"Debug - Arguments: {tool_call.function.arguments}")

    try:
        tool_arguments = json.loads(tool_call.function.arguments)
        tool_query_string = tool_arguments.get('statement', '')
    except json.JSONDecodeError:
        print(f"Error: Unable to parse function arguments: {tool_call.function.arguments}")
        tool_query_string = ''

    # Step 3: Call the function and retrieve results. Append the results to the messages list.
    if tool_function_name == 'shout':
        results = shout(tool_query_string)

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call_id,
            "name": tool_function_name,
            "content": results
        })

        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f"Error: function {tool_function_name} does not exist")
else:
    # Model did not identify a function to call, result can be returned to the user
    print(response_message.content)

Debug - Tool call received: shout
Debug - Arguments: {"statement":"We are there"}
WE ARE THERE! How can I assist you further?
